In [1]:
import cv2
import numpy as np
from math import acos, pi, sqrt, cos

In [2]:
# B,G,R
def compute_Hue(B, G, R):
    angle = 0
    if B != G != R:
        angle = 0.5*((R - G) + (R - B)) / sqrt((R-G)*(R-G) + (R-B)*(G-B))
        if B<= G:
            return acos(angle)
        elif B> G:
            return (2*pi - acos(angle))
    #return acos(angle) if B <= G else (2*pi - acos(angle))
    
def compute_BGR(I,S,H):    
    R = np.zeros((height, width,3), dtype=np.float32)*255
    G = np.zeros((height, width,3), dtype=np.float32)*255
    B = np.zeros((height, width,3), dtype=np.float32)*255
    
    if H >= 0 and H < 120: #0도~120도
        B = (1-S)*I*255
        R = (1+(S*cos(H) / cos(60-H)))*I*255
        G = (I*3 - (R+B))*255
    elif H >= 120 and H < 240:
        H = H-120
        R = (1-S)*I/3*255
        G = (1+(S*cos(H) / cos(60-H)))*I*255
        B = (I*3 - (R+G))*255
        
    elif H >= 240 and H <= 360:
        H = H - 240
        G = (1-S)*I*255
        B = (1+(S*cos(H) / cos(60-H)))*I*255
        R = (I*3 - (G+B))*255
        
    B = np.asarray(B, dtype=np.uint8)*255
    G = np.asarray(G, dtype=np.uint8)*255
    R = np.asarray(R, dtype=np.uint8)*255
    return [B,G,R]

In [ ]:
src = cv2.imread('test image/bridge.bmp', cv2.IMREAD_COLOR)
height, width = src.shape[0], src.shape[1]

I = np.zeros((height, width))
S = np.zeros((height, width))
H = np.zeros((height, width))
hsi2B = np.zeros((height, width))
hsi2G = np.zeros((height, width))
hsi2R = np.zeros((height, width))


hsi2BGR = np.zeros((height, width))

B, G, R = cv2.split(src)
#B = src[:,:,0]
#G = src[:,:,1]
#R = src[:,:,2]


for i in range(height):
    for j in range(width):
        B_tmp, G_tmp, R_tmp = B[i][j]/255., G[i][j]/255., R[i][j]/255.
        
        I[i][j] = (B_tmp+G_tmp+R_tmp)/3.
        if B_tmp+G_tmp+R_tmp != 0:
            S[i][j] = 1 - 3*np.min([B_tmp,G_tmp,R_tmp])/(B_tmp+G_tmp+R_tmp)
        H[i][j] = compute_Hue(B_tmp, G_tmp, R_tmp)

        
negative = np.zeros((height, width,3), dtype=np.uint8)
plt_1 = np.zeros((height, width,3), dtype=np.uint8)
plt_2 = np.zeros((height, width,3), dtype=np.uint8)
plt_3 = np.zeros((height, width,3), dtype=np.uint8)
plt_4 = np.zeros((height, width,3), dtype=np.uint8)


I = I*255
S = S*255
H = H*255/360

I = np.asarray(I, dtype=np.uint8)
S = np.asarray(S, dtype=np.uint8)
H = np.asarray(H, dtype=np.uint8)


#Negative
for i in range(height):
    for j in range(width):
        [hsi2B[i][j], hsi2G[i][j], hsi2R[i][j]] = compute_BGR(I[i][j], S[i][j], H[i][j])



negative[:,:,0] = hsi2B
negative[:,:,1] = hsi2G
negative[:,:,2] = hsi2R   



#Negative
for i in range(height):
    for j in range(width):
        negative[i][j] = -src[i][j]

#power-law transformation : lambda value = 0.20
for i in range(height):
    for j in range(width):
        plt_1[i][j] = ((src[i][j]/255.)**0.20)*255
            
#power-law transformation : lambda value = 0.40
for i in range(height):
    for j in range(width):
        plt_2[i][j] = ((src[i][j]/255.)**0.40)*255.
        
#power-law transformation : lambda value = 1.5
for i in range(height):
    for j in range(width):
        plt_3[i][j] = ((src[i][j]/255.)**1.5)*255.
        
#power-law transformation : lambda value = 5.0
for i in range(height):
    for j in range(width):
        plt_4[i][j] = ((src[i][j]/255.)**5.0)*255.

        


hsi2B = np.asarray(hsi2B, dtype=np.uint8)/255.
hsi2G = np.asarray(hsi2G, dtype=np.uint8)/255.
hsi2R = np.asarray(hsi2R, dtype=np.uint8)/255.        
        
#cv2.imshow("H", H)
#cv2.imshow("S", S)
#cv2.imshow("I", I)
#cv2.imshow("R", R)
#cv2.imshow("G", G)
#cv2.imshow("B", B)
#cv2.imshow("hsi2R", hsi2R)
#cv2.imshow("hsi2G", hsi2G)
#cv2.imshow("hsi2B", hsi2B)
cv2.imshow("negative", negative)
cv2.imshow("lambda 0.2", plt_1)
cv2.imshow("lambda 0.4", plt_2)
cv2.imshow("lambda 1.5", plt_3)
cv2.imshow("lambda 5.0", plt_4)
cv2.imshow("src", src)
cv2.waitKey(0)
cv2.destroyAllWindows()